In [2]:
%%bash

# Use the spark_version variable from the notebook
SPARK_VERSION="${spark_version#spark-}"

# Update Homebrew and install dependencies
brew update
brew install openjdk apache-spark wget

# Set environment variables
echo 'export JAVA_HOME="/usr/local/opt/openjdk"' >> ~/.zshrc
echo 'export SPARK_HOME="/usr/local/Cellar/apache-spark/'"$SPARK_VERSION"'/libexec"' >> ~/.zshrc
echo 'export PATH="$JAVA_HOME/bin:$SPARK_HOME/bin:$PATH"' >> ~/.zshrc
source ~/.zshrc

# Verify installation
echo "Java Version:"
java -version

echo "Spark Version:"
pyspark --version

echo "Setup complete! 🚀"


==> Updating Homebrew...


Already up-to-date.


To reinstall 23.0.2, run:
  brew reinstall openjdk
To reinstall 3.5.5, run:
  brew reinstall apache-spark


==> Downloading https://ghcr.io/v2/homebrew/core/wget/manifests/1.25.0
==> Fetching dependencies for wget: libidn2
==> Downloading https://ghcr.io/v2/homebrew/core/libidn2/manifests/2.3.8
==> Fetching libidn2
==> Downloading https://ghcr.io/v2/homebrew/core/libidn2/blobs/sha256:ef2c85790a55f54f58fceb849e1ee46f2b06ebe516c83043e68c979e47934e6f
==> Fetching wget
==> Downloading https://ghcr.io/v2/homebrew/core/wget/blobs/sha256:5650778a8e7a60c2dea9412dd21d2f5e8ff4f224dbefbdf54924b99012062edc
==> Installing dependencies for wget: libidn2
==> Installing wget dependency: libidn2
==> Downloading https://ghcr.io/v2/homebrew/core/libidn2/manifests/2.3.8
Already downloaded: /Users/GURU/Library/Caches/Homebrew/downloads/f30f50fbde4bff9a71de54d684e482d7da3432656d680b97441163c6e5665468--libidn2-2.3.8.bottle_manifest.json
==> Pouring libidn2--2.3.8.sonoma.bottle.tar.gz
🍺  /usr/local/Cellar/libidn2/2.3.8: 80 files, 838.2KB
==> Installing wget
==> Pouring wget--1.25.0.sonoma.bottle.tar.gz
🍺  /usr/loca

bash: line 10: /Users/GURU/.zshrc: Permission denied
bash: line 11: /Users/GURU/.zshrc: Permission denied
bash: line 12: /Users/GURU/.zshrc: Permission denied


Java Version:


java version "1.8.0_371"
Java(TM) SE Runtime Environment (build 1.8.0_371-b11)
Java HotSpot(TM) 64-Bit Server VM (build 25.371-b11, mixed mode)


Spark Version:


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.5
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 17.0.15
Branch HEAD
Compiled by user ubuntu on 2025-02-23T20:30:46Z
Revision 7c29c664cdc9321205a98a14858aaf8daaa19db2
Url https://github.com/apache/spark
Type --help for more information.


Setup complete! 🚀


In [3]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
import time
import datetime
from pyspark.sql.functions import year, to_date, regexp_replace

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 11:58:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# revise original column data formats:
# id remains unrevised
df = df.withColumn("year_sold", year(df['date']))
# date remains unrevise
df = df.withColumn("date_built", year(df['date_built']))
df = df.withColumn("price", df["price"].cast(IntegerType()))
df = df.withColumn("bedrooms", df["bedrooms"].cast(IntegerType()))
df = df.withColumn("bathrooms", df["bathrooms"].cast(IntegerType()))
df = df.withColumn("sqft_living", df["sqft_living"].cast(IntegerType()))
df = df.withColumn("sqft_lot", df["sqft_lot"].cast(IntegerType()))
df = df.withColumn("floors", df["floors"].cast(IntegerType()))
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|     2022|


In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)
 |-- year_sold: integer (nullable = true)



In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
select year_sold, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
where bedrooms==4
group by year_sold
order by year_sold DESC
"""
).show()

+---------+---------+
|year_sold| avgPrice|
+---------+---------+
|     2022|296363.88|
|     2021|301819.44|
|     2020|298353.78|
|     2019|300263.70|
+---------+---------+



In [8]:
# 4. What is the average price of a home for each date the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
select date_built, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
where bedrooms==3 and bathrooms==3
group by date_built
order by date_built DESC
"""
).show()


+----------+---------+
|date_built| avgPrice|
+----------+---------+
|      2017|292676.79|
|      2016|290555.07|
|      2015|288770.30|
|      2014|290852.27|
|      2013|295962.27|
|      2012|293683.19|
|      2011|291117.47|
|      2010|292859.62|
+----------+---------+



In [9]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
select date_built as year_built, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
where bedrooms==3 and bathrooms==3 and floors==2 and sqft_living>= 2000
group by date_built
order by date_built DESC
"""
).show()


+----------+---------+
|year_built| avgPrice|
+----------+---------+
|      2017|280317.58|
|      2016|293965.10|
|      2015|297609.97|
|      2014|298264.72|
|      2013|303676.79|
|      2012|307539.97|
|      2011|276553.81|
|      2010|285010.22|
+----------+---------+



In [10]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
select DISTINCT view, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
group by view
having avgPrice >= 350000
order by view DESC
"""
).show(100)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95|1054325.60|
|  94|1033536.20|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87|1072285.20|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82|1063498.00|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74| 745077.00|
|  73| 752861.18|
|  72| 780914.67|
|  71| 775651.10|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66| 712475.00|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59| 791453.00|
|  58| 759764.65|
|  57| 734340.50|
|  56| 718176.40|
|  55| 771153.32|
|  54| 798684.82|
|  53| 755214.80|
|  52| 733780.26|
|  51| 788128.21|
| 100|1026669.50|
+----+----------+

--- 0.6328651905059814 seco

In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
select DISTINCT view, cast(AVG(price) as numeric (9,2)) as avgPrice
from home_sales
group by view
having avgPrice >= 350000
order by view DESC
"""
).show(100)


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95|1054325.60|
|  94|1033536.20|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87|1072285.20|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82|1063498.00|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74| 745077.00|
|  73| 752861.18|
|  72| 780914.67|
|  71| 775651.10|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66| 712475.00|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59| 791453.00|
|  58| 759764.65|
|  57| 734340.50|
|  56| 718176.40|
|  55| 771153.32|
|  54| 798684.82|
|  53| 755214.80|
|  52| 733780.26|
|  51| 788128.21|
| 100|1026669.50|
+----+----------+

--- 0.40729713439941406 sec

In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('home_sales',mode='overwrite')

In [15]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('home_sales')

In [16]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
select DISTINCT view, cast(AVG(price) as numeric (9,2)) as avgPrice
from p_home_sales
group by view
having avgPrice >= 350000
order by view DESC
"""
).show(100)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  avgPrice|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95|1054325.60|
|  94|1033536.20|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87|1072285.20|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82|1063498.00|
|  81|1053472.79|
|  80| 991767.38|
|  79|1009565.08|
|  78|1080649.37|
|  77|1076205.56|
|  76|1058802.78|
|  75|1114042.94|
|  74| 745077.00|
|  73| 752861.18|
|  72| 780914.67|
|  71| 775651.10|
|  70| 695865.58|
|  69| 750537.94|
|  68| 716785.44|
|  67| 737970.96|
|  66| 712475.00|
|  65| 736679.93|
|  64| 767036.67|
|  63| 711614.55|
|  62| 759150.14|
|  61| 746877.59|
|  60| 754939.65|
|  59| 791453.00|
|  58| 759764.65|
|  57| 734340.50|
|  56| 718176.40|
|  55| 771153.32|
|  54| 798684.82|
|  53| 755214.80|
|  52| 733780.26|
|  51| 788128.21|
| 100|1026669.50|
+----+----------+

--- 0.8713531494140625 seco

In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False